In [2]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from matplotlib import pyplot
from keras.layers.embeddings import Embedding
from keras.layers import Flatten, Dense, LSTM, SimpleRNN
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.preprocessing import sequence
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import numpy as np

Using TensorFlow backend.


In [3]:
top_words = 5000
max_words = 500

#### Get the imdb_data setting max words to include to 5000

In [4]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = top_words)

In [6]:
x_train = sequence.pad_sequences(x_train, maxlen=100)
x_test = sequence.pad_sequences(x_test, maxlen=100)

In [7]:
x_train

array([[1415,   33,    6, ...,   19,  178,   32],
       [ 163,   11, 3215, ...,   16,  145,   95],
       [1301,    4, 1873, ...,    7,  129,  113],
       ...,
       [  11,    6, 4065, ...,    4, 3586,    2],
       [ 100, 2198,    8, ...,   12,    9,   23],
       [  78, 1099,   17, ...,  204,  131,    9]], dtype=int32)

#### Concatenate data the train/test data

In [20]:
data = np.concatenate((x_train, x_test), axis=0)
target = np.concatenate((y_train, y_test), axis=0)

#### Print the first review

In [32]:
print("Review: ", data[0]), print("Label: ", target[0])

Review:  [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]
Label:  1


(None, None)

#### Retreive the dictionary mapping

In [33]:
index = imdb.get_word_index()

In [35]:
pos = dict([(value, key) for (key, value) in index.items()])

In [37]:
decoded = " ".join([pos.get(i-3, "#") for i in data[0]])
print(decoded)

# this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert # is an amazing actor and now the same being director # father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for # and would recommend it to everyone to watch and the fly # was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also # to the two little # that played the # of norman and paul they were just brilliant children are often left out of the # list i think because the stars that play them all grown up are such a big # for the whole film but these children are amazing and should be # for what they have done don't you think the whole story was

#### Pad all the reviews so that their length is max_words

In [1]:
data = sequence.pad_sequences(data, maxlen=max_words)

NameError: name 'sequence' is not defined

#### Train/Test Split so we have 20% for Validation

In [41]:
data.shape

(50000, 500)

In [48]:
datasize = data.shape[0]
testsize = int(datasize*.2)
print(testsize)

10000


In [49]:
x_test = data[:testsize]
y_test = target[:testsize]
x_train = data[testsize:]
y_train = target[testsize:]

#### Params for Embedding: 
Dictate the input layer size: Output of embedding will be 500 vectors (max_words) of 32 dimensions (embedding_vector_length) each.  This output will get flattened to 500 *32 element vector for the next layer to receive.

In [51]:
embedding_vector_length = 32

In [52]:
top_words, embedding_vector_length, max_words 

(5000, 32, 500)

### Plotting Utitlity function to compare accuracy during training/validation

In [ ]:
def plot(history):
    

### MLP

In [53]:
model = Sequential()
embedding_vector_length = 32
model.add(Embedding(top_words, embedding_vector_length, input_length=max_words))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               2048128   
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 2,224,769
Trainable params: 2,224,769
Non-trainable params: 0
_________________________________________________________________
None


In [54]:
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=128, verbose=2)
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %f" % (scores[1]*100))

/opt/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 40000 samples, validate on 10000 samples
Epoch 1/10
 - 13s - loss: 0.3946 - accuracy: 0.7974 - val_loss: 0.2785 - val_accuracy: 0.8841
Epoch 2/10
 - 13s - loss: 0.1574 - accuracy: 0.9431 - val_loss: 0.3343 - val_accuracy: 0.8699
Epoch 3/10
 - 13s - loss: 0.0483 - accuracy: 0.9852 - val_loss: 0.5175 - val_accuracy: 0.8607
Epoch 4/10


KeyboardInterrupt: 

### LSTM 

In [11]:
embedding_vector_length = 32
model1 = Sequential()
model1.add(Embedding(top_words, embedding_vector_length, input_length=max_words))
model1.add(LSTM(100))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
model1.fit(x_train, y_train, epochs=3, batch_size=128, verbose=2)
# Final evaluation of the model

Epoch 1/3
 - 902s - loss: 0.3014 - accuracy: 0.8787
Epoch 2/3
 - 884s - loss: 0.2307 - accuracy: 0.9110
Epoch 3/3
 - 878s - loss: 0.2080 - accuracy: 0.9204


In [14]:
scores = model1.evaluate(x_test, y_test, verbose=0)

In [16]:
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 85.40%


### CNN

In [17]:
embedding_vector_length = 32
model2 = Sequential()
model2.add(Embedding(top_words, embedding_vector_length, input_length=max_words))
model2.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
model2.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
model2.add(Flatten())
#model2.add(Dense(128, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
model2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 498, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 249, 32)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 247, 32)           3104      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 123, 32)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3936)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                

In [19]:
model2.fit(x_train, y_train, epochs=3, batch_size=128, verbose=2)

/Users/michael/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
 - 70s - loss: 0.5617 - accuracy: 0.6606
Epoch 2/3
 - 57s - loss: 0.2593 - accuracy: 0.8951
Epoch 3/3
 - 57s - loss: 0.1959 - accuracy: 0.9260


In [20]:
scores = model2.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 87.78%


### RNN

In [22]:
embedding_vector_length = 32
model3 = Sequential()
model3.add(Embedding(top_words, embedding_vector_length, input_length=max_words))
model3.add(SimpleRNN(100))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model3.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 100)               13300     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 173,401
Trainable params: 173,401
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
model3.fit(x_train, y_train, epochs=3, batch_size=128, verbose=2)

/Users/michael/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
 - 215s - loss: 0.6768 - accuracy: 0.5598
Epoch 2/3
 - 208s - loss: 0.6223 - accuracy: 0.6663
Epoch 3/3
 - 205s - loss: 0.5652 - accuracy: 0.7030


In [24]:
scores = model3.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 69.04%
